In [1]:
from datasets import load_dataset
import random
from transformers import pipeline
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
#from tqdm import tqdm
from tqdm.notebook import tqdm
import evaluate

In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
dataset = load_dataset('code-search-net/code_search_net')

README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

java.zip:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

javascript.zip:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

...

ruby.zip:   0%|          | 0.00/112M [00:00<?, ?B/s]

php.zip:   0%|          | 0.00/852M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1880853 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100529 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/89154 [00:00<?, ? examples/s]

In [4]:
sample_df = pd.DataFrame(dataset['train'])
sample_df.columns

Index(['repository_name', 'func_path_in_repository', 'func_name',
       'whole_func_string', 'language', 'func_code_string', 'func_code_tokens',
       'func_documentation_string', 'func_documentation_tokens', 'split_name',
       'func_code_url'],
      dtype='object')

In [8]:
train_df = pd.read_excel("self_training_sample_Fsoft.xlsx", sheet_name="Sheet2", usecols=[5, 10, 12, 14])
train_df.columns

Index(['language', 'docstring', 'code', 'short_docstring'], dtype='object')

In [9]:
train_df.head()

,language,docstring,code,short_docstring
0,C,iperf_time_compare\nCompare two timestamps\n\n...,int\niperf_time_compare(struct iperf_time *tim...,iperf_time_compare\n Compare two timestamps
1,C,Search a table for an entry with matching valu...,"static int npu_table_search(struct npu2 *p, ui...",Search a table for an entry with matching valu...
2,C,Conditionally reduce the per-virtual-corner ce...,static int cpr_reduce_ceiling_voltage(struct c...,Conditionally reduce the per-virtual-corner ce...
3,C,The DMA API client is passing in a scatterlist...,"int iommu_dma_map_sg(struct device *dev, struc...",The DMA API client is passing in a scatterlist...
4,C,"Initialize a tunnel instance, handle pre and p...",void\ninit_instance_handle_signals (struct con...,"Initialize a tunnel instance, handle pre and p..."


In [20]:
#dataset_sample = random.choices(dataset['test'], k=50)
dataset_sample = []
while len(dataset_sample) < 50:
    message_string = random.choices(dataset['test'], k=1)
    full_message = message_string[0]['func_documentation_string'] + message_string[0]['func_code_string']
    if len(full_message) / 4 < 4000:
        dataset_sample.append(message_string)

# Testing the Baseline model

Here we define the system prompt for the Llama 2 model.

In [16]:
prompt_llama = \
'''You are a helpful agent designed to simplify code documentation for beginner programmers.
You will be provided with a block of code and the existing doucmentation that accompanies it.
Simplify the given documentation, using the provided code as context, so that it is understandable
to beginner programmers. Output absolutely nothing else besides the simplified documentation.
Make sure to keep any documentation formatting codes present in the simplified documentation.
If you feel that the existing documentation is simple enough and meaning would be lost by simplifying
it further, feel free to keep the documentation as is. Here is the original documentation and code:'''

Creating the pipeline for the Llama 2 model using the HuggingFace transformers library. Modified from the example here: https://huggingface.co/docs/transformers/en/model_doc/llama2

In [14]:
pipe_llama = pipeline(
    task="text-generation",
    model="meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.float16,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda


In [18]:
eval_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
message = [
    {"role": "system", "content": prompt_llama},
    {"role": "user", "content": f"Documentation:\n{dataset['test'][0]['func_documentation_string']}\n\nCode:\n{dataset['test'][0]['func_code_string']}"}
]
print(f"Original Documentation:\n{dataset['test'][0]['func_documentation_string']}\n")
print(f"Code:\n{dataset['test'][0]['func_code_string']}\n")
print(pipe_llama(message, pad_token_id=pipe_llama.tokenizer.eos_token_id)[0]['generated_text'][-1]['content'])

Original Documentation:
Extracts video ID from URL.

Code:
def get_vid_from_url(url):
        """Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')

  Simplified Documentation:

This function extracts the video ID from a URL. It uses a combination of regular expressions and query parameter parsing to extract the video ID from various parts of the URL.

Here are the steps it follows:

1. Checks if the URL contains the "youtu.be" domain and extracts the video ID from the URL using a regular expression.
2. Checks if the URL contains the "youtube.com/embed" or "youtube.com/v" domains and extracts the video ID from the URL using regular expressions.
3. Checks if the URL cont

In [21]:
llama_semantic_similarities_untrained = []
llama_metrics_untrained = evaluate.combine(['rouge', 'meteor'])

for instance in tqdm(dataset_sample):
    message = [
        {"role": "system", "content": prompt_llama},
        {"role": "user", "content": f"Documentation:\n{instance[0]['func_documentation_string']}\n\nCode:\n{instance[0]['func_code_string']}"}
    ]

    result = pipe_llama(message, pad_token_id=pipe_llama.tokenizer.eos_token_id)[0]['generated_text'][-1]['content']

    embedding_original = eval_model.encode(instance[0]['func_documentation_string'], convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    llama_semantic_similarities_untrained.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    llama_metrics_untrained.add(predictions=result, references=instance[0]['func_documentation_string'])

[nltk_data] Downloading package wordnet to /home/adeniji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/adeniji/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/adeniji/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


  0%|          | 0/50 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [22]:
for index, row in train_df.iterrows():
    print(row['language'])

C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#
C#


In [ ]:
def return_prompt(language):
    return \
    f'''You are a helpful agent designed to simplify code documentation for beginner programmers.
    You will be provided with a block of {language} code and the existing doucmentation that accompanies it.
    Simplify the given documentation, using the provided code as context, so that it is understandable
    to beginner programmers. Output absolutely nothing else besides the simplified documentation.
    Make sure to keep any documentation formatting codes present in the simplified documentation.
    If you feel that the existing documentation is simple enough and meaning would be lost by simplifying
    it further, feel free to keep the documentation as is. Here is the original documentation and code:'''

In [ ]:
llama_semantic_similarities_untrained = []
llama_metrics_untrained = evaluate.combine(['rouge', 'meteor'])

for index, row in train_df.iterrows():
    prompt_llama_training = return_prompt(row['language'])

    message = [
        {"role": "system", "content": prompt_llama},
        {"role": "user", "content": f"Documentation:\n{row['docstring']}\n\nCode:\n{row['code']}"}
    ]

    result = pipe_llama(message, pad_token_id=pipe_llama.tokenizer.eos_token_id)[0]['generated_text'][-1]['content']

    embedding_original = eval_model.encode(row['short_docstring'], convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    llama_semantic_similarities_untrained.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    llama_metrics_untrained.add(predictions=result, references=row['short_docstring'])
   

# Summary statistics results
Semantic Similarity

In [ ]:
print("Untrained Llama 2: \n")
pd.DataFrame(llama_semantic_similarities_untrained).describe()
print("Trained Llama 2: \n")
pd.DataFrame(llama_semantic_similarities_trained).describe()

ROUGE AND METEOR

In [ ]:
llama_metrics_untrained.compute()